In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))


In [2]:

from stable_diffusion.pipeline import generate
from PIL import Image
from transformers import CLIPTokenizer
import torch
from stable_diffusion.utils.model_loader import preload_models_from_standard_weights


/home/groups/ai/gupta/anaconda/anaconda3/envs/diff/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# DEVICE = "cpu"  # Forcing CPU usage
print(f"Using device: {DEVICE}")


Using device: cuda


In [4]:
tokenizer = CLIPTokenizer("../dataa/vocab.json", merges_file="../dataa/merges.txt")
model_file = "../dataa/v1-5-pruned-emaonly.ckpt"
models = preload_models_from_standard_weights(model_file, DEVICE)

In [13]:

# text to image
prompt = "fuck you"
do_cfg = True
cfg_scale=20

#image to image

image_path = "../image/A-Cat.jpg"
input_image = Image.open(image_path)

strength = 1 # between 0 and 1

sampler = "ddpm"
num_inference_steps = 60
seed = 42

output_image = generate(
    prompt = prompt,
    uncon_prompt = "should not be cartoon, painting, drawing, lowres",
    strength = strength,
    do_cfg = do_cfg,
    cfg_scale = cfg_scale,
    sampler_name= sampler,
    n_inference_steps = num_inference_steps,
    seed = seed,
    device = DEVICE,
    idle_device="cpu",
    model = models,
    tokenizer = tokenizer,
    video = False,
    input_image = None,


)


100%|██████████| 60/60 [00:04<00:00, 12.55it/s]
